In [238]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [239]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from functions import *

pd.set_option('display.max_columns', None)

df_merged = pd.read_csv("data/df_merged_national.csv")

In [240]:
df_merged = clean_df(df_merged)
df_merged = every_float_to_int(df_merged)

In [241]:
df_no_provincias = df_merged[(df_merged["region"] == "    DESCONOCIDA") | (df_merged["region"] == "    EN EL EXTRANJERO") | (df_merged["region"] == "    TOTAL NACIONAL")]

df_no_provincias.to_csv("data/df_no_provincias.csv", index=False)

In [242]:
df_merged = df_merged[~df_merged.index.isin(df_no_provincias.index)]

In [243]:
df_num = df_merged.iloc[:, df_merged.columns != 'region']

In [244]:
df_merged = add_coordinates_from_dict(df_merged)

In [245]:
regions = {
    "ANDALUCÍA": 0,
    "ARAGÓN": 1,
    "ASTURIAS (PRINCIPADO DE)": 2,
    "BALEARS (ILLES)": 3,
    "CANTABRIA": 4,
    "CANARIAS": 5,
    "CASTILLA - LA MANCHA": 6,
    "CASTILLA Y LEÓN": 7,
    "CATALUÑA": 8,
    "CIUDAD AUTÓNOMA DE CEUTA": 9,
    "CIUDAD AUTÓNOMA DE MELILLA": 10,
    "COMUNITAT VALENCIANA": 11,
    "EXTREMADURA": 12,
    "GALICIA": 13,
    "MADRID (COMUNIDAD DE)": 14,
    "MURCIA (REGIÓN DE)": 15,
    "NAVARRA (COMUNIDAD FORAL DE)": 16,
    "PAÍS VASCO": 17,
    "RIOJA (LA)": 18
}


df_merged["region"] = df_merged["region"].replace(regions)


C:\Users\VSPC\AppData\Local\Temp\ipykernel_15484\2512969448.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged["region"] = df_merged["region"].replace(regions)


In [246]:
df_merged.insert(0, "year", df_merged.pop("year"))

In [247]:
df_merged["avg_crimes"] = df_merged.iloc[:, 2:-4].mean(axis=1)      # [:, 2:-2] quiero -> todas las filas, desde la 2da hasta la penultima columna

In [248]:
df_merged = delete_sub_crimes(df_merged)

In [249]:
df_merged["years_from_today"] = 2024 - df_merged["year"]

In [250]:
df_poblacion = pd.read_csv("data/cleaned_pop.csv")


df_poblacion.drop(["Nacional", "Sexo"], inplace=True, axis=1)

regions_pop_version = {
    "01 Andalucía": 0,
    "02 Aragón": 1,
    "03 Asturias Principado de": 2,
    "04 Balears Illes": 3,
    "06 Cantabria": 4,
    "05 Canarias": 5,
    "08 Castilla - La Mancha": 6,
    "07 Castilla y León": 7,
    "09 Cataluña": 8,
    "18 Ceuta": 9,
    "19 Melilla": 10,
    "10 Comunitat Valenciana": 11,
    "11 Extremadura": 12,
    "12 Galicia": 13,
    "13 Madrid Comunidad de": 14,
    "14 Murcia Región de": 15,
    "15 Navarra Comunidad Foral de": 16,
    "16 País Vasco": 17,
    "17 Rioja La": 18
}


df_poblacion["region"] = df_poblacion["region"].replace(regions_pop_version)

C:\Users\VSPC\AppData\Local\Temp\ipykernel_15484\1471067258.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_poblacion["region"] = df_poblacion["region"].replace(regions_pop_version)


In [251]:
df_poblacion

,region,year,Total
0,0,2019,8.414.240
1,0,2018,8.384.408
2,0,2017,8.379.820
3,0,2016,8.388.107
4,0,2015,8.399.043
...,...,...,...
185,10,2014,84.509
186,10,2013,83.679
187,10,2012,80.802
188,10,2011,78.476


In [252]:
df_merged = df_merged.merge(df_poblacion, on=["region", "year"])

df_merged.rename(columns={"Total": "population"}, inplace=True)

In [253]:
df_merged

,year,region,1_against_people,2_against_freedom,3_sexual_freedom,4_family_relationships,5_property_crimes,6_collective_security,7_forgeries,8_public_administration,9_justice_administration,latitude,longitude,avg_crimes,years_from_today,population
0,2019,0,37130,25751,2510,1503,239711,15094,3988,93,6728,37.3873,-5.9869,17723.219512,5,8.414.240
1,2019,1,4760,2276,330,161,31651,1360,446,3,799,41.6488,-0.8891,2249.902439,5,1.319.291
2,2019,2,3368,1872,193,99,19066,1205,220,6,567,43.3619,-5.8494,1429.390244,5,1.022.800
3,2019,3,8110,4718,743,188,58274,2880,716,11,1095,39.5712,2.6466,4082.146341,5,1.149.460
4,2019,5,12396,9328,907,639,64716,3439,1114,17,1721,28.2916,-16.6291,4960.829268,5,2.153.389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,2010,16,2726,675,130,82,13770,1059,259,21,235,42.6954,-1.6761,977.414634,14,636.924
186,2010,17,11792,4257,417,337,65555,3241,543,25,1170,43.2630,-2.9349,4515.878049,14,2.178.339
187,2010,18,1695,170,49,31,6555,519,54,0,137,42.2871,-2.5396,484.804878,14,322.415
188,2010,9,833,277,18,31,3053,1381,53,2,101,35.8894,-5.3198,300.804878,14,80.579


In [254]:
crime_values = ['1_against_people', '2_against_freedom',
       '3_sexual_freedom', '4_family_relationships', '5_property_crimes',
       '6_collective_security', '7_forgeries', '8_public_administration',
       '9_justice_administration']

In [255]:
df_unpivot = pd.melt(df_merged, id_vars=['year', 'region', 'latitude', 'longitude', 'population'], value_vars=crime_values)

df_unpivot

,year,region,latitude,longitude,population,variable,value
0,2019,0,37.3873,-5.9869,8.414.240,1_against_people,37130
1,2019,1,41.6488,-0.8891,1.319.291,1_against_people,4760
2,2019,2,43.3619,-5.8494,1.022.800,1_against_people,3368
3,2019,3,39.5712,2.6466,1.149.460,1_against_people,8110
4,2019,5,28.2916,-16.6291,2.153.389,1_against_people,12396
...,...,...,...,...,...,...,...
1705,2010,16,42.6954,-1.6761,636.924,9_justice_administration,235
1706,2010,17,43.2630,-2.9349,2.178.339,9_justice_administration,1170
1707,2010,18,42.2871,-2.5396,322.415,9_justice_administration,137
1708,2010,9,35.8894,-5.3198,80.579,9_justice_administration,101


In [256]:
df_unpivot_transformed = df_unpivot.copy()

for crime in crime_values:
    column_name = f'{crime}_indicator'
    df_unpivot_transformed[column_name] = (df_unpivot_transformed['variable'] == crime).astype(int)

df_unpivot_transformed

,year,region,latitude,longitude,population,variable,value,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator
0,2019,0,37.3873,-5.9869,8.414.240,1_against_people,37130,1,0,0,0,0,0,0,0,0
1,2019,1,41.6488,-0.8891,1.319.291,1_against_people,4760,1,0,0,0,0,0,0,0,0
2,2019,2,43.3619,-5.8494,1.022.800,1_against_people,3368,1,0,0,0,0,0,0,0,0
3,2019,3,39.5712,2.6466,1.149.460,1_against_people,8110,1,0,0,0,0,0,0,0,0
4,2019,5,28.2916,-16.6291,2.153.389,1_against_people,12396,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,2010,16,42.6954,-1.6761,636.924,9_justice_administration,235,0,0,0,0,0,0,0,0,1
1706,2010,17,43.2630,-2.9349,2.178.339,9_justice_administration,1170,0,0,0,0,0,0,0,0,1
1707,2010,18,42.2871,-2.5396,322.415,9_justice_administration,137,0,0,0,0,0,0,0,0,1
1708,2010,9,35.8894,-5.3198,80.579,9_justice_administration,101,0,0,0,0,0,0,0,0,1


In [257]:
df_values = get_previous_years(df_unpivot_transformed)

df_values 

,year,region,latitude,longitude,population,variable,value,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,values_year_minus_1,values_year_minus_2,values_year_minus_3
0,2019,0,37.3873,-5.9869,8.414.240,1_against_people,37130,1,0,0,0,0,0,0,0,0,35043.0,35087.0,33661.0
1,2019,1,41.6488,-0.8891,1.319.291,1_against_people,4760,1,0,0,0,0,0,0,0,0,4474.0,4146.0,4005.0
2,2019,2,43.3619,-5.8494,1.022.800,1_against_people,3368,1,0,0,0,0,0,0,0,0,3159.0,3160.0,3239.0
3,2019,3,39.5712,2.6466,1.149.460,1_against_people,8110,1,0,0,0,0,0,0,0,0,7545.0,7096.0,6609.0
4,2019,5,28.2916,-16.6291,2.153.389,1_against_people,12396,1,0,0,0,0,0,0,0,0,11805.0,11744.0,11021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2013,16,42.6954,-1.6761,644.477,9_justice_administration,198,0,0,0,0,0,0,0,0,1,234.0,230.0,235.0
1193,2013,17,43.2630,-2.9349,2.191.682,9_justice_administration,1470,0,0,0,0,0,0,0,0,1,1458.0,1357.0,1170.0
1194,2013,18,42.2871,-2.5396,322.027,9_justice_administration,115,0,0,0,0,0,0,0,0,1,147.0,141.0,137.0
1195,2013,9,35.8894,-5.3198,84.180,9_justice_administration,125,0,0,0,0,0,0,0,0,1,157.0,84.0,101.0


In [258]:
df_total_values = df_values.dropna(subset=['values_year_minus_1', 'values_year_minus_2', 'values_year_minus_3']).reset_index(drop=True)

df_total_values

,year,region,latitude,longitude,population,variable,value,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,values_year_minus_1,values_year_minus_2,values_year_minus_3
0,2019,0,37.3873,-5.9869,8.414.240,1_against_people,37130,1,0,0,0,0,0,0,0,0,35043.0,35087.0,33661.0
1,2019,1,41.6488,-0.8891,1.319.291,1_against_people,4760,1,0,0,0,0,0,0,0,0,4474.0,4146.0,4005.0
2,2019,2,43.3619,-5.8494,1.022.800,1_against_people,3368,1,0,0,0,0,0,0,0,0,3159.0,3160.0,3239.0
3,2019,3,39.5712,2.6466,1.149.460,1_against_people,8110,1,0,0,0,0,0,0,0,0,7545.0,7096.0,6609.0
4,2019,5,28.2916,-16.6291,2.153.389,1_against_people,12396,1,0,0,0,0,0,0,0,0,11805.0,11744.0,11021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2013,16,42.6954,-1.6761,644.477,9_justice_administration,198,0,0,0,0,0,0,0,0,1,234.0,230.0,235.0
1193,2013,17,43.2630,-2.9349,2.191.682,9_justice_administration,1470,0,0,0,0,0,0,0,0,1,1458.0,1357.0,1170.0
1194,2013,18,42.2871,-2.5396,322.027,9_justice_administration,115,0,0,0,0,0,0,0,0,1,147.0,141.0,137.0
1195,2013,9,35.8894,-5.3198,84.180,9_justice_administration,125,0,0,0,0,0,0,0,0,1,157.0,84.0,101.0


In [259]:
# Tabla final

#df_total_values.to_csv('df_total_values.csv', index=False)

df_total_values['population'] = df_total_values['population'].str.replace('.', '', regex=False).astype(float)




### Aplicar PCA para las columnas

In [260]:
columns_pca = ['values_year_minus_1', 'values_year_minus_2', 'values_year_minus_3']
values_year_data = df_total_values[columns_pca]

scaler = StandardScaler()
values_year_scaled = scaler.fit_transform(values_year_data)

pca = PCA(n_components=0.9, random_state=42)
pca_result = pca.fit_transform(values_year_scaled)

pca_column = [f'Value_year_PCA{i+1}' for i in range(pca_result.shape[1])]
df_pca_values = pd.DataFrame(pca_result, columns=pca_column)

df_pca_values

,Value_year_PCA1
0,0.862743
1,-0.305943
2,-0.345329
3,-0.195319
4,-0.024596
...,...
1192,-0.458865
1193,-0.416703
1194,-0.462375
1195,-0.463435


In [261]:
# Merge

df_total_pca_values = df_total_values.copy()

df_total_pca_values['Values_year_PCA'] = df_pca_values['Value_year_PCA1']

df_total_pca_values = df_total_pca_values.drop(columns=columns_pca)

df_total_pca_values.to_csv('df_total_pca_values.csv', index=False)

In [262]:
df_total_values_201017 = df_total_pca_values[df_total_values['year'] <= 2017]

df_total_values_201819 = df_total_pca_values[df_total_values['year'] >= 2018]

In [263]:
features = [
    'year','latitude', 'longitude', 'population',
    '1_against_people_indicator', '2_against_freedom_indicator',
    '3_sexual_freedom_indicator', '4_family_relationships_indicator',
    '5_property_crimes_indicator', '6_collective_security_indicator',
    '7_forgeries_indicator', '8_public_administration_indicator',
    '9_justice_administration_indicator', 'Values_year_PCA'
    ]

X_train = df_total_values_201017[features]
y_train = df_total_values_201017['value']

X_test = df_total_values_201819[df_total_values_201819['year'] == 2018][features]
y_test = df_total_values_201819[df_total_values_201819['year'] == 2018]['value']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((855, 14), (855,), (171, 14), (171,))

In [264]:
df_total_values_201017[features]

,year,latitude,longitude,population,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,Values_year_PCA
38,2017,37.3873,-5.9869,8379820.0,1,0,0,0,0,0,0,0,0,1.333843
39,2017,41.6488,-0.8891,1308750.0,1,0,0,0,0,0,0,0,0,-0.279516
40,2017,43.3619,-5.8494,1034960.0,1,0,0,0,0,0,0,0,0,-0.302724
41,2017,39.5712,2.6466,1115999.0,1,0,0,0,0,0,0,0,0,-0.164682
42,2017,28.2916,-16.6291,2108121.0,1,0,0,0,0,0,0,0,0,0.122110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2013,42.6954,-1.6761,644477.0,0,0,0,0,0,0,0,0,1,-0.458865
1193,2013,43.2630,-2.9349,2191682.0,0,0,0,0,0,0,0,0,1,-0.416703
1194,2013,42.2871,-2.5396,322027.0,0,0,0,0,0,0,0,0,1,-0.462375
1195,2013,35.8894,-5.3198,84180.0,0,0,0,0,0,0,0,0,1,-0.463435


### Random forest sin estandarizar y normalizar

In [265]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

In [266]:
rf_reg = RandomForestRegressor(
    n_estimators=300, 
    max_leaf_nodes=None,
    min_samples_split=5,
    n_jobs=-1, 
    random_state=42
    )

rf_reg.fit(X_train, y_train)
y_pred = rf_reg.predict(X_test)



In [267]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 1033.0388923532055
Root Mean Squared Error (RMSE): 4707.907926358284
R² Score: 0.9896681902859178


### Estcalado

In [268]:
features_nn = [
    'year','latitude', 'longitude', 'population',
    '1_against_people_indicator', '2_against_freedom_indicator',
    '3_sexual_freedom_indicator', '4_family_relationships_indicator',
    '5_property_crimes_indicator', '6_collective_security_indicator',
    '7_forgeries_indicator', '8_public_administration_indicator',
    '9_justice_administration_indicator', 'Values_year_PCA'
    ]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[features_nn])
X_test_scaled = scaler.transform(X_test[features_nn])

rf_reg_nn = RandomForestRegressor(
    n_estimators=300, 
    max_leaf_nodes=None,
    min_samples_split=5,
    n_jobs=-1, 
    random_state=42
)

rf_reg_nn.fit(X_train_scaled, y_train)
y_pred = rf_reg_nn.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred) 
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:}")
print(f"Root Mean Squared Error (RMSE): {rmse:}")
print(f"R² Score: {r2:}")

Mean Absolute Error (MAE): 1035.4595720898653
Root Mean Squared Error (RMSE): 4709.648084016559
R² Score: 0.9896605510963612


### For loop

In [269]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

X_train = df_total_values_201017[features]
y_train = df_total_values_201017['value']

X_test = df_total_values_201819[df_total_values_201819['year'] == 2018][features]
y_test = df_total_values_201819[df_total_values_201819['year'] == 2018]['value']

rf_reg = RandomForestRegressor(
    n_estimators=300, 
    max_leaf_nodes=None,
    min_samples_split=5,
    n_jobs=-1, 
    random_state=42
    )

rf_reg.fit(X_train, y_train)

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")


Mean Absolute Error (MAE): 1035.4595720898653
Root Mean Squared Error (RMSE): 4709.648084016559
R² Score: 0.9896605510963612


In [270]:
# Predecir 2017
# ->Predecir 2018
# ->Predecir 2019

future_years = [2018, 2019, 2020, 2021, 2022]

df_try_1 = df_total_values_201017.copy()

for year in future_years:
    X_year = df_try_1[df_try_1['year'] == year - 1][features_nn]

    y_pred_year = rf_reg.predict(X_year)

    new_year_df = df_try_1[df_try_1['year']==year-1].copy()
    new_year_df['year'] = year
    new_year_df['value'] = y_pred_year

    df_try_1 = pd.concat([df_try_1, new_year_df], ignore_index=True)

    y_year = df_total_values_201819[df_total_values_201819['year'] == year]['value']

    if not y_year.empty:
        mae = mean_absolute_error(y_year, y_pred_year)
        rmse = root_mean_squared_error(y_year, y_pred_year)
        r2 = r2_score(y_year, y_pred_year)

        print(f"{year}")
        print(f"Mean Absolute Error (MAE): {mae}")
        print(f"Root Mean Squared Error (RMSE): {rmse}")
        print(f"R² Score: {r2}")


2018
Mean Absolute Error (MAE): 862.4179010140823
Root Mean Squared Error (RMSE): 4669.712778372217
R² Score: 0.9898351537080551
2019
Mean Absolute Error (MAE): 1214.0161715530508
Root Mean Squared Error (RMSE): 6140.436835969422
R² Score: 0.9833668676167556


### Apply RandomSearch

In [271]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, root_mean_squared_error

X_train = df_total_values_201017[features]
y_train = df_total_values_201017['value']

X_test = df_total_values_201819[df_total_values_201819['year'] == 2018][features]
y_test = df_total_values_201819[df_total_values_201819['year'] == 2018]['value']

rf_base = RandomForestRegressor(random_state=42)

rf_params = {
    'n_estimators': [100, 300, 500, 800, 1000, 2000],          
    'max_features': ['auto', 'sqrt', 'log2'],            
    'max_depth': [None, 10, 20, 30, 40, 50],              
    'min_samples_split': [2, 5, 10],                      
    'min_samples_leaf': [1, 2, 4],                        
    'bootstrap': [True, False]                            
    }

random_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=rf_params,
    n_iter=50,  
    scoring='neg_mean_squared_error',  
    cv=3,  
    verbose=2,  
    random_state=42,
    n_jobs=-1
    )

random_search.fit(X_train, y_train)

print(f"Best hyperparameters: {random_search.best_params_}")


y_pred_test = random_search.best_estimator_.predict(X_test)
test_mse = root_mean_squared_error(y_test, y_pred_test)

print(f"Best model: {test_mse}")


Fitting 3 folds for each of 50 candidates, totalling 150 fits


c:\Users\VSPC\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
36 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
21 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\VSPC\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\VSPC\anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\VSPC\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\VSPC\anaconda3\Lib\site-packages\skle

Best hyperparameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
Best model: 4369.038006154812


In [272]:
future_years = [2018, 2019, 2020, 2021, 2022]

best_rf_model = random_search.best_estimator_

df_total_predict_201822 = df_total_values_201017.copy()

for year in future_years:
    X_year = df_total_predict_201822[df_total_predict_201822['year'] == year - 1][features_nn]

    y_pred_year = best_rf_model.predict(X_year)

    new_year_df = df_total_predict_201822[df_total_predict_201822['year']==year-1].copy()
    new_year_df['year'] = year
    new_year_df['value'] = y_pred_year

    df_total_predict_201822 = pd.concat([df_total_predict_201822, new_year_df], ignore_index=True)

    y_year = df_total_values_201819[df_total_values_201819['year'] == year]['value']

    if not y_year.empty:
        mae = mean_absolute_error(y_year, y_pred_year)
        rmse = root_mean_squared_error(y_year, y_pred_year)
        r2 = r2_score(y_year, y_pred_year)

        print(f"{year}")
        print(f"Mean Absolute Error (MAE): {mae}")
        print(f"Root Mean Squared Error (RMSE): {rmse}")
        print(f"R² Score: {r2}")

2018
Mean Absolute Error (MAE): 676.5521978557512
Root Mean Squared Error (RMSE): 4197.781233500472
R² Score: 0.9917858981983638
2019
Mean Absolute Error (MAE): 1073.8309990253417
Root Mean Squared Error (RMSE): 5660.856101274083
R² Score: 0.9858635702934668


### Apply to all for 2020-2022 using PCA per step

In [273]:
columns_pca = ['values_year_minus_1', 'values_year_minus_2', 'values_year_minus_3']
df_total_values = df_unpivot_transformed

df_total_values['population'] = df_total_values['population'].str.replace('.', '', regex=False).astype(float)

df_total_values = get_previous_years(df_total_values)

df_total_values_PCA = apply_pca(df_total_values, columns_pca)

X_train = df_total_values_PCA[features]
y_train = df_total_values_PCA['value']

X_test = df_total_values_PCA[df_total_values_PCA['year'] == 2019][features]

df_total_values_PCA


,year,region,latitude,longitude,population,variable,value,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,Values_year_PCA
0,2019,0,37.3873,-5.9869,8414240.0,1_against_people,37130,1,0,0,0,0,0,0,0,0,0.862743
1,2019,1,41.6488,-0.8891,1319291.0,1_against_people,4760,1,0,0,0,0,0,0,0,0,-0.305943
2,2019,2,43.3619,-5.8494,1022800.0,1_against_people,3368,1,0,0,0,0,0,0,0,0,-0.345329
3,2019,3,39.5712,2.6466,1149460.0,1_against_people,8110,1,0,0,0,0,0,0,0,0,-0.195319
4,2019,5,28.2916,-16.6291,2153389.0,1_against_people,12396,1,0,0,0,0,0,0,0,0,-0.024596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2013,16,42.6954,-1.6761,644477.0,9_justice_administration,198,0,0,0,0,0,0,0,0,1,-0.458865
1193,2013,17,43.2630,-2.9349,2191682.0,9_justice_administration,1470,0,0,0,0,0,0,0,0,1,-0.416703
1194,2013,18,42.2871,-2.5396,322027.0,9_justice_administration,115,0,0,0,0,0,0,0,0,1,-0.462375
1195,2013,9,35.8894,-5.3198,84180.0,9_justice_administration,125,0,0,0,0,0,0,0,0,1,-0.463435


In [286]:
df_total_predict_202021 = df_total_values_PCA.copy()

future_years = [2020, 2021]

for year in future_years:

    #Drop PCA var
    df_total_predict_202021 = df_total_predict_202021.drop(columns=['Values_year_PCA'], errors='ignore')

    #Recalculate year_minus
    df_total_predict_202021 = get_previous_years(df_total_predict_202021)

    #Apply PCA
    df_total_predict_202021 = apply_pca(df_total_predict_202021, columns_pca)

    #Predict next year
    X_year = df_total_predict_202021[df_total_predict_202021['year'] == year - 1][features_nn]
    y_pred_year = best_rf_model.predict(X_year)

    new_year_df = df_total_predict_202021[df_total_predict_202021['year'] == year - 1].copy()
    new_year_df['year'] = year
    new_year_df['value'] = y_pred_year

    df_total_predict_202021 = pd.concat([df_total_predict_202021, new_year_df], ignore_index=True)

    y_year = df_total_values[df_total_values['year'] == year]['value']

df_total_predict_202021



,year,region,latitude,longitude,population,variable,value,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,Values_year_PCA
0,2019,0,37.3873,-5.9869,8414240.0,1_against_people,37130.000000,1,0,0,0,0,0,0,0,0,0.870682
1,2019,1,41.6488,-0.8891,1319291.0,1_against_people,4760.000000,1,0,0,0,0,0,0,0,0,-0.298335
2,2019,2,43.3619,-5.8494,1022800.0,1_against_people,3368.000000,1,0,0,0,0,0,0,0,0,-0.337385
3,2019,3,39.5712,2.6466,1149460.0,1_against_people,8110.000000,1,0,0,0,0,0,0,0,0,-0.187941
4,2019,5,28.2916,-16.6291,2153389.0,1_against_people,12396.000000,1,0,0,0,0,0,0,0,0,-0.016985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,2021,16,42.6954,-1.6761,654214.0,9_justice_administration,488.069722,0,0,0,0,0,0,0,0,1,-0.445348
509,2021,17,43.2630,-2.9349,2207776.0,9_justice_administration,1659.554167,0,0,0,0,0,0,0,0,1,-0.402499
510,2021,18,42.2871,-2.5396,316798.0,9_justice_administration,261.753611,0,0,0,0,0,0,0,0,1,-0.453035
511,2021,9,35.8894,-5.3198,84777.0,9_justice_administration,271.358611,0,0,0,0,0,0,0,0,1,-0.453465


In [275]:
df_total_predict_202021.to_csv('df_total_predict_202021.csv')